## **3. 협업 필터링 추천 시스템(Collaborative Filtering, CF)**
### 3.1 협업 필터링의 원리
- 취향이 비슷한 사람들의 집단이 존재한다고 가정
- 한 사람과 취향이 비슷한 사람들(=neighbor)을 찾음
- 이 사람들이 공통적으로 좋아하는 제품/서비스를 추천 대상인 사람에게 추천하면 됨

예) 영화 추천<br>
- 5개의 영화(M1 ~ M5)
- 5명의 사용자(U1 ~ U5)
- 영화 재미도에 대해 1~5점 만점으로 평가

#### CF 방법을 사용한 영화 추천
1. U1과 취향이 비슷한 사용자 찾아내기<br>
: 각 사용자의 평가의 유사성(similarity) 계산, 상관계수 계산 ▶ 높은 상관관계 보이면 취향이 비슷한 neighbor 그룹으로 분류
<br>

2. U3, U4가 가장 좋게 평가한 영화 찾아내기<br>
: U1이 M4, M5를 아직 보지 않았으므로 U1과 가장 유사한 U3, U4의 M4, M5에 대한 평점 평균은 각각 2,5임
<br>

3. M5를 U1에게 추천 영화로 제시

### 3.2 유사도지표(similarity index)
: 사용자들의 평가를 기반으로 사용자 간 유사도 구하는 지표

#### 1) 상관계수(Correlation coefficient)
: 평가 자료가 연속값인 경우 가장 이해하기 쉬운 유사도 
- 최소 -1(완전 반대) ~ 최대 +1(완전 일치)까지의 값을 가짐
- 협업 필터링에 사용하는 경우 항상 좋은 결과를 가져오지는 못함

#### 2) 코사인 유사도(Cosine similarity)
: 각 아이템을 하나의 차원(dimension)으로 보고, 사용자의 평가값을 좌표값으로 봄
- 각 사용자의 평가값 = 벡터, 두 사용자간의 벡터의 각도(코사인값) 구하는 방식
- 두 아이템(A,B), 두 사용자(x,y) ▶ 원점에서 사용자까지의 선을 그으면 사용자의 벡터 형성
- 두 사용자의 평가값이 유사할수록 각도가 작음(코사인값이 큼)
- 최소 -1(완전 불일치) ~ 최대 +1(완전 일치)까지의 값을 가짐
<br>

데이터가 이진값(binary: 0과 1로만 이루어짐)이면 상관계수/코사인 유사도 사용할 수 없음.

#### 3) 타니모토 계수(Tanimoto coefficient)
##### simil(x,y) = c / (a + b - c)
<br>

- a: 사용자 x가 1의 값을 갖는(구입한/클릭한) 아이템의 수
- b: 사용자 y가 1의 값을 갖는(구입한/클릭한) 아이템의 수
- c: 사용자 x와 y가 공통적으로 1의 값을 갖는 아이템의 수

- 최소 0(완전 불일치) ~ 최대 +1(완전 일치)까지의 값을 가짐
- 이진수 데이터에 대해 협업 필터링에서 좋은 결과를 구해줌
- 자카드 계수(Jaccard coefficient)

### 3.3 기본 CF 알고리즘
: 현재 사용자와 취향이 비슷한 사용자 그룹 따로 선정하지 않고 모든 사용자의 평점을 가지고 예측 (=이웃(neighbor)을 전체사용자로 함)

#### CF 알고리즘 작동 방식
1. 모든 사용자간의 평가의 유사도 계산 - 유사도 지표 중 하나 선택
2. 현재 추천 댓ㅇ이 되는 사람과 다른 사람의 유사도 추출
3. 현재 사용자가 평가하지 않은 모든 아이템에 대해 현재 사용자의 예상 평가값 구함
- 예상 평가값: 다른 사용자의 해당 아이템에 대한 평가를 유사도로 가중해 평균내는 방식
<br>
4. 아이템 중 예상 평가값이 가장 높은 N개 아이템 추천

In [2]:
#코사인 유사도 사용

#2단원에서 사용한 함수 그대로 가져와 사용
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')


# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)


# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)


# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [5]:
#train set의 모든 가능한 사용자 pair의 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity #cosine similiarity 계산하는 라이브러리

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

#### 함수 정의&설명
- matrix_dummy = rating_matrix.copy().fillna(0)<br>
: 코사인 유사도를 계산하기 위해 rating의 matrix 값을 matrix_dummy에 복사, NaN값을 0으로 대체
<br>

- user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)<br>
: 모든 사용자 간의 코사인 유사도 구함 ▶ user_similarity는 943* 943 차원, 대각선 1인 대칭 매트릭스
<br>

- user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)<br>
: user_similarity에 인덱스 지정

In [6]:
#주어진 영화(movie_id)의 가중평균 rating 계산하는 함수 정의
#가중치: 주어진 사용자와 다른 사용자간의 유사도(user_similarity)
    
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

score(CF_simple)
#평가경향이 비슷한(=유사도가 높은) 사용자에게 가중치를 더 주어 평균 계산시 더 정확한 예측할 수 있음

1.0217783221096222

#### 함수 정의&설명
- sim_scores = user_similarity[user_id].copy()<br>
: 주어진 사용자(user_id)와 다른 사용자의 유사도 복사
<br>

- none_rating_idx = movie_ratings[movie_ratings.isnull()].index<br>
: 주어진 영화에 대해 평가를 하지 않은 사용자의 위치 확인 ▶ 가중평균 계산에서 제외
<br>

- movie_ratings = movie_ratings.dropna()<br>
: 주어진 영화에 대한 평점 중 평가하지 않은 사람의 평점 제거(NaN으로 표시되어 있는 사람)
<br>

- sim_scores = sim_scores.drop(none_rating_idx)<br>
: 영화 평가하지 않은 사용자와의 유사도 제거(가중평균 계산시 필요하지 않음)

### 3.4 이웃을 고려한 CF
: 이웃(neighbor)을 전체 사용자로 하여 유사도가 높은 사람만을 이웃으로 선정 ▶ 이웃의 크기 줄이는 것 
- 대상 사용자와 유사도가 높은 사람의 평가만 사용하면 예측 정확도 상승

#### 이웃을 정하는 기준
1. K Nearest Neighbors(KNN)<br>
: 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 상위 K명을 선택하는 방법

2. Thresholding<br>
: 크기 대신 유사도의 기준(예: 상관계수 0.8 이상, S)을 정해놓고 이 기준을 충족시키는 사용자를 숫자 관계없이 이웃으로 정하는 방법

In [10]:
#KNN 사용(neighbor_size = 0)
#neighbor_size = 0: 전체 사용자 데이터를 다 사용하는 앞의 일반적인 CF

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')


# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)


# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)


# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


# train set 사용자들의 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [12]:
#Neighbor size를 정해서 예측치를 계산하는 함수
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(none_rating_idx)
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        
#Neighbor size가 지정되지 않은 경우
        if neighbor_size == 0:          
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

#Neighbor size가 지정된 경우
        else:                       
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1: 
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                
                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 영화 rating을 neighbor size만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산 
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating
    
score(cf_knn, neighbor_size = 30)
#simple_CF()보다 RMSE가 약간 더 개선되었음

1.005908425238956

#### 함수 정의&설명
- neighbor_size = min(neighbor_size, len(sim_scores))<br>
: 지정된 neighbor_size 값과 해당 영화를 평가한 총 사용자 수 중에서 작은 것을 이웃 크기로 설정<br>
: 경우에 따라 유사도 계산이 가능한 이웃의 수가 지정된 neighbor_size보다 작을 수도 있음
<br>

- sim_scores = sim_scores[user_idx][-neighbor_size:]<br>
: 선정된 k명의 사용자의 주어진 영화에 대한 평점 추출
<br>

- mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()<br>
: 선정된 k명의 이웃 사용자의 해당 영화에 대한 평점을 유사도로 가중한 평점평균 추출

In [13]:
#주어진 사용자에 대해 추천받기 (추천 대상 user_id, 추천 아이템 수, 이웃크기[neighbor_size]) ▶ 추천 영화 리스트

#전체 데이터로 full matrix, 코사인 유사도 구하기
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)


def recom_movie(user_id, n_items, neighbor_size=30):
    # 현 사용자가 평가한 영화 가져오기
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

recom_movie(user_id=2, n_items=5, neighbor_size=30)

movie_id
1293                     Ayn Rand: A Sense of Life (1997)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1189                              That Old Feeling (1997)
1467                                     Cure, The (1995)
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

#### 함수 정의&설명
- if pd.notnull(user_movie.loc[movie]): user_movie.loc[movie] = 0<br>
: 현재 영화 평점이 null이 아닌 경우, 사용자가 이미 이 영화를 평가했으므로 추천 리스트에서 제외 ▶ 예상 평점 = 0
<br>

- else: user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)<br>
: 영화 평점이 null인 경우 현 사용자가 평가하지 않음 ▶ cf_knn 함수 호출해 예상 평점 계산
<br>

- movie_sort = user_movie.sort_values(ascending=False)[:n_items]<br>
: 영화 예상 평점 기준으로 영화 리스트 내림차순 정렬, 상위 영화 추천 아이템 수(n_items)만큼 추출

### 3.5 최적의 이웃 크기 결정
: 일반적으로 이웃 크기가 증가하면서 추천의 정확도는 증가하다가 어느 시점 이후에 감소하는 형태를 보임
- 이웃의 크기가 지나치게 크면: 집단별 취향 차이 없어짐, 정확도 떨어짐, 개인 취향 반영 안됨
- 이웃의 크기가 지나치게 작으면: 현재 사용자와 취향(평가 패턴)의 유사도가 높은 소수의 이웃의 평가만 사용하게 됨, 지나치게 의존하며 예측치의 신뢰성 낮아짐 ▶ 과적합(over-fitting)
<br>

#### 과적합
: 주어진 train 데이터에 너무 딱 맞도록 학습을 해서 실제 데이터(test set)에 대해 예측할 때에는 정확도가 떨어지는 현상

In [14]:
#최적의 neighbor size 구하기

#train set으로 full matrix, 코사인 유사도 구하기
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [15]:
#neighbor_size가 10~60인 경우에 대해 RMSE 계산, 출력
for neighbor_size in [10,20,30,40,50,60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0232
Neighbor size = 20 : RMSE = 1.0090
Neighbor size = 30 : RMSE = 1.0059
Neighbor size = 40 : RMSE = 1.0064
Neighbor size = 50 : RMSE = 1.0067
Neighbor size = 60 : RMSE = 1.0074


### 3.6 사용자의 평가경향을 고려한 CF
: 사용자의 평가경향(user bias)를 고려해 예측치 조정하는 방법<br>
예) 평가를 전체적으로 높게 하는 사람/평가를 전체적으로 낮게 하는 사람

<b>예상 평점:</b><br>
[(사용자와 이웃 유사도의 전체 합) * (아이템 i에 대한 사용자의 평점)] / (사용자와 이웃 유사도의 전체 합)

#### 평가경향 고려한 CF 알고리즘 작동 방식
1. 각 사용자 평점평균 구함
2. 각 아이템의 평점을 각 사용자의 평균에서의 차이 (평균 - 사용자의 평점평균)으로 변환 ▶ scaling 작업
- 평점편차: (평균 - 사용자의 평점평균)
3. 해당 사용자의 해당 아이템의 평점편차 예측값 구함<br>
- 이웃 구한 뒤 이웃의 해당 아이템에 대한 평점편차, 유사도 가중평균
4. 현 사용자의 평균에 이 편차 예측값 더해줌
5. 예측값을 구할 수 없는 경우 중간값인 3.0이 아닌, 사용자의 평점평균으로 대체

In [16]:
#KNN 사용(neighbor_size = 0)
#neighbor_size = 0: 전체 사용자 데이터를 다 사용하는 앞의 일반적인 CF

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')


# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)


# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)


# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


# train set 사용자들의 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [18]:
#모든 유저의 rating 평균, 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis = 1) #평점평균
rating_bias = (rating_matrix.T - rating_mean).T #평점 - 평균 = 평점편차

In [19]:
def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
#Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]

#Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산            
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

score(CF_knn_bias, 30)

0.9457692698703326

#### 함수 정의&설명
- movie_ratings = rating_bias[movie_id].copy()<br>
: 해당 영화의 평점 대신 평점편차를 복사
<br>

- prediction = prediction + rating_mean[user_id]<br>
: 예측치를 평점편차로 구했으므로 최종 예측값은 해당 사용자의 평균을 더해줌
<br>

- else: prediction = rating_mean[user_id]<br>
: 예측치를 정상적으로 구할 수 없는 경우에는 3.0 대시 사용자의 평점평균으로 계산

### 연습문제
#### 3-2 사용자ID 지정했을 때 5개의 영화 추천하도록 함

In [20]:
def recom_movie2(user_id, n_items, neighbor_size=30):
    # 현 사용자가 평가한 영화 가져오기
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = CF_knn_bias(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

recom_movie2(user_id=2, n_items=5, neighbor_size=30)

movie_id
1449                               Golden Earrings (1947)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1293                     Ayn Rand: A Sense of Life (1997)
1612                                    Tokyo Fist (1995)
1368    Forbidden Christ, The (Cristo proibito, Il) (1...
Name: title, dtype: object

### 3.7 그 외의 CF 정확도 개선 방법
1. 신뢰도 가중(significance weighting)<br>
: 공통 아이템이 많은 사용자와의 유사도에 공통 아이템이 적은 사용자와의 유사도보다 더 큰 가중치를 주는 방법
- 예측값이 매우 민갑해 RMSE값 크게 변동 
- 공통 아이템의 수를 가중치로 직접 사용하면 문제 발생 ▶ 신뢰도가 일정 이상인 사용자만을 예측치 계산에 사용
- 신뢰도(공통으로 평가한 아이템의 수)가 일정값 이상인 사용자만 이웃 사용자로 활용함

<br>
2. 예측값 계산하는 대상 아이템에 대해 평가한 사용자가 일정 수준 이상이 되는 아이템만 예측치를 계산하도록 함

- 평가한 사용자가 2명 이상인 경우만 계산 ▶ 일반화하여 n명 이상인 경우만 계산

In [21]:
#KNN 사용(neighbor_size = 0)
#neighbor_size = 0: 전체 사용자 데이터를 다 사용하는 앞의 일반적인 CF

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')


# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)


# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)


# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


# train set 사용자들의 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

In [22]:
#모든 유저의 rating 평균, 영화의 평점편차 계산
rating_mean = rating_matrix.mean(axis = 1) #평점평균
rating_bias = (rating_matrix.T - rating_mean).T #평점 - 평균 = 평점편차

In [23]:
#사용자별 공통 평가 수 계산
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)


def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 True로 표시
        
        low_significance = common_counts < SIG_LEVEL
        
        #평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)

#Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]

#Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction


SIG_LEVEL = 3 #최소 신뢰도(공통 평가 영화 수) = 3
MIN_RATINGS = 2 #최소 사용자 수(영화 평가한 사용자 수) = 2
score(CF_knn_bias_sig, 30)

0.9457189433261078

#### 함수 정의&설명
- rating_binary1 = np.array((rating_matrix > 0).astype(float))<br>
: 전체 full matrix 중에서 평점이 있는 경우만 1로 표시
<br>

- rating_binary2 = rating_binary1.T<br>
: 위에서 구한 1로 표시한 행렬의 전치행렬(transposed matrix) 구함
<br>

- counts = np.dot(rating_binary1, rating_binary2)<br>
: 두 행렬의 내적 연산(dot)를 해 각 원소가 각 사용자가 공통으로 평가한 영화의 수, 대각선은 각 사용자가 평가한 영화의 수

### 3.8 사용자 기반 CF(UBCF) & 아이템 기반 CF(IBCF)
- <b>사용자 기반 CF (User-Based CF)</b><br>
: 취향이 비슷한 이웃 사용자(neighbor)을 알아내는 방식 ▶ 그룹에 속한 사용자들이 공통적으로 좋게 평가한 아이템 추천
<br>
    1) 장점: 사용자별 맞춤형 추천이기에 데이터가 많으면 정확한 추천<br>
    2) 단점: 데이터가 조금 바뀔때마다 업데이트 해야함

- <b>아이템 기반 CF (Item-Based CF)</b><br>
: 사용자들의 평가 패턴 바탕으로 아이템 간 유사도 계산, 사용자의 특정 아이템에 대한 예측 평점 계산하는 방식
<br>
1) 장점: 사용자별로 따로따로 계산하지 않기에 계산이 빠름, 업데이트 자주 안해줘도 됨

✅데이터 크기가 적고 각 사용자에 대한 충분한 정보(구매.평가)가 있는 경우 ▶ UBCF<br>
✅데이터 크기가 크고 각 사용자에 대한 충분한 정보가 없는 경우 ▶ IBCF

In [28]:
#KNN 사용(neighbor_size = 0)
#neighbor_size = 0: 전체 사용자 데이터를 다 사용하는 앞의 일반적인 CF

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')


# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]


# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)


# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


# 모델별 RMSE를 계산하는 함수 (neighbor_size 정의하는 부분 생략)
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기  
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [29]:
# train set의 모든 가능한 아이템 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

#### 함수 정의&설명
- item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)<br>
: 아이템 간의 코사인 유사도를 item_similarity에 저장 (=아이템간의 유사도행렬)

In [30]:
# IBCF(아이템 기반 협업 필터링 추천 시스템)

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_IBCF)

1.0195470744904096

### 3.9 추천 시스템의 성과측정지표
#### 데이터 2개 세트로 나누는 과정
1. 데이터를 train set, test set으로 나눔
- train set: 추천 모형을 학습시키는 데이터
- test set: 정확도를 계산하기 위한 데이터

2. train set을 이용해 test set의 추천 만들어냄
3. 예측값과 test set의 실제값 차이를 계산해 정확도 계산

#### 정확도 측정방법
1. 각 아이템의 예측값 - 실제값 차이 계산하는 방법 (연속값의 경우)
- MAD(Mean Absolute Deviation)
- MSE(Mean Squared Error)
- RMSE(Root Mean Squared Error)

2. 추천 아이템과 사용자의 실제 선택 비교하는 방법 (이진값의 경우)
- 정밀도(precision)
- 재현율(recall)
- 정확도(accuracy)
- 정밀도와 재현율의 조화평균(F1 score)
- 범위(coverage)

---
💡<b>정확도(accuracy)</b> = (올바르게 예측된 아이템의 수) / (전체 아이템의 수) <br>
: 사용자가 아이템을 선택했는지뿐만 아니라 선택하지 않았다는 것을 올바르게 예측한 비율
<br>

💡<b>정밀도(precision)</b> = (올바르게 추천된 아이템의 수) / (전체 추천된 아이템의 수)<br>
: 추천 시스템이 추천한 아이템 중 실제 사용자가 선택한 아이템의 비율
<br>

💡<b>재현율(recall)</b> = (올바르게 추천된 아이템의 수) / (사용자가 실제로 선택한 전체 아이템의 수)<br>
: 사용자가 실제로 선택한 전체 아이템 중 몇 개의 아이템을 올바르게 맞췄는가<br>
- 정밀도와 재현율은 서로 반대의 관계(trade-off) 
- 동시에 고려하는 조화평균(F1 score) 자주 사용
<br>
<br>


💡<b>정밀도와 재현율의 조화평균(F1 score)</b> = (2 * 정밀도 * 재현율) / (정밀도 + 재현율)
<br>

💡<b>범위(coverage)</b> = (추천이 가능한 사용자 수(아이템 수)) / (전체 사용자 수(아이템 수))<br>
- 정밀도와 반대의 관계

#### True Positive Rate(TPR) & False Positive Rate(FPR)
- TPR: 정확하게 추천된 아이템의 비율(사용자가 실제로 선택한 모든 아이템 중 추천 시스템이 제대로 맞게 추천한 아이템의 비율)
- FPR: 잘못 추천된 아이템의 비율(사용자가 선택하지 않은 아이템 중 추천 시스템이 추천한 아이템의 비율)

#### TP, FP, FN, TN
- Accuracy = (TP+TN)/(TP+TN+FP+FN)
- Precision = TP/(TP+FP)
- TPR = Recall = TP/(TP+FN)
- FPR = FP/(FP+TN)